In [1]:
from core.utils import Tibanna
from dcicutils import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'data'
tibanna = Tibanna(env=env)

ff = ff_utils.fdn_connection(key={"default" : tibanna.ff_keys})
exclude_miseq = True

In [6]:
import time

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

out_n = "This is an output file of the Hi-C processing pipeline"
int_n = "This is an intermediate file in the HiC processing pipeline"

def step_settings(seq, my_organism):
    genome = ""
    mapper = {'human':'GRCh38','mouse':'GRCm38'}
    genome = mapper.get(my_organism)
    
    wf_dict =[{
        'wf_name': 'bwa-mem',
        'wf_uuid': '3feedadc-50f9-4bb4-919b-09a8b731d0cc',
        'parameters': {"nThreads": 16},
        'custom_pf_fields': {
            'out_bam': {
                'genome_assembly': genome,
                'file_type': 'intermediate file',
                'description': int_n}
        }},
        {
        'wf_name': 'hi-c-processing-bam',
        'wf_uuid': '023bfb3e-9a8b-42b9-a9d4-216079526f68',
        'parameters': {"nthreads_merge": 16, "nthreads_parse_sort": 16},
        'custom_pf_fields': {
            'annotated_bam': {
                'genome_assembly': genome,
                'file_type': 'alignment',
                'description': out_n},
            'filtered_pairs': {
                'genome_assembly': genome,
                'file_type': 'contact list-replicate',
                'description': out_n}
        }},
        {
        'wf_name': 'hi-c-processing-pairs-nore',
        'wf_uuid': 'c19ee11e-9d5a-454f-af50-600a0cf990b6',
        'parameters': {"nthreads": 1, "maxmem": "32g"},
        'custom_pf_fields': {
            'cooler_normvector': {
                'genome_assembly': genome,
                'file_type': 'juicebox norm vector',
                'description': out_n},
            'hic': {
                'genome_assembly': genome,
                'file_type': 'contact matrix',
                'description': out_n},
            'mcool': {
                'genome_assembly': genome,
                'file_type': 'contact matrix',
                'description': out_n},
            'merged_pairs': {
                'genome_assembly': genome,
                'file_type': 'contact list-combined',
                'description': out_n}
        }}]
    
    return wf_dict[seq]


# url for micro-C Dnase exps
exp_types = ['micro-C', 'DNase%20Hi-C']
set_url = '/search/?'+ \
            '&'.join(['experiments_in_set.experiment_type='+i for i in exp_types])+ \
            '&type=ExperimentSetReplicate&limit=all' + \
            '&status=released&status=released%20to%20project'

run_sets = ff_utils.search_metadata(set_url , ff_env=env)

add_pc = True
add_tag = True

add_rel = False
add_wfr = False


counter = 0
completed = 0
completed_acc = []

all_sets = len(run_sets)
print(str(all_sets)+' total number of sets')

run_sets = [i for i in run_sets if "HiC_Pipeline_0.2.5"  not in i.get('completed_processes', [])]
print(str(all_sets-len(run_sets))+ ' sets completed')

for a_set in run_sets: 
    counter += 1

    print 
    fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size, lab = find_pairs(a_set, exclude_miseq, env, tibanna)
    
    
    
    print f_size

    
    
    
    if not bwa_ref or not chrsize_ref:
        print counter, a_set['accession'], organism, enzyme, 'skipping set with not chrsize/bwa index'
        continue
    
    if f_size < 15:
        print counter, a_set['accession'], 'skipping small file size', str(f_size) 
        continue
        
        
    print counter, a_set['accession']
    print enzyme, organism
    part3 = 'done'
    list_release = []
    set_pairs = []        
    # cycle through the experiments
    for exp in fastqpairs.keys():
        if not fastqpairs.get(exp):
            print(exp, 'does not have any fastq pairs')
            continue
        # Check Part 1 and See if all are okay
        exp_bams = []
        part1 = 'done'
        part2 = 'done'
        for pair in fastqpairs[exp]:
            #############
            bam1 = get_wfr_out(pair[0], 'bwa-mem 0.2.5', 'bam', env)
            bam2 = get_wfr_out(pair[1], 'bwa-mem 0.2.5', 'bam', env)
            # if run is not successful
            if bam1.startswith('no') or not bam1 or bam1 != bam2:
                part1 = 'not ready'
                if add_wfr:
                    if not bwa_index:
                        print 'not yet usable', organism
                        continue
                    inp_f = {'fastq1':pair[0], 'fastq2':pair[1], 'bwa_index':bwa_ref}
                    name_tag = pair[0].split('/')[2]+'_'+pair[1].split('/')[2]
                    run_missing_wfr(step_settings(0, organism), inp_f, name_tag, env, tibanna)
            elif bam1 == 'running':
                part1 = 'still running'
                print('part1 still running')
            # if successful
            else:
                exp_bams.append(bam1)
                list_release.append(bam1)
        # stop progress to part2 
        if part1 is not 'done':
            print exp, 'has missing Part1 runs'
            part2 = 'not ready'
            part3 = 'not ready'
            continue
        print exp, 'part1 complete'
        #check if part 2 is run already, it not start the run
        exp_com_bam = []
        exp_pairs = []
        for bam in exp_bams:
            com_bam = get_wfr_out(bam, 'hi-c-processing-bam 0.2.5', 'bam', env)
            pairs = get_wfr_out(bam, 'hi-c-processing-bam 0.2.5', 'pairs', env)
            # try to run if missing
            if pairs.startswith('no') or not pairs:
                part2 = 'not ready'
                part3 = 'not ready'
                
            elif pairs == 'running':
                part2 = 'still running'
                part3 = 'not ready'
                
            else:
                exp_com_bam.append(com_bam)
                exp_pairs.append(pairs)
                
        # if still running, skip to next experiment
        if part2 == 'still running':
            print('part2 still running')
            continue
        
        # make sure all bams went through the same wfr and produces same file
        if part2 != 'done' or len(list(set(exp_com_bam))) != 1 or len(list(set(exp_pairs))) !=1:
            print exp, 'Part2 did not complete'
            part3 = 'not ready' 
        
            if add_wfr:
                if not chrsize_ref:
                    print 'not yet usable', organism
                    continue
                # make sure no duplicates
                inp_f = {'input_bams':exp_bams, 'chromsize':chrsize_ref}           
                run_missing_wfr(step_settings(1, organism), inp_f, exp, env, tibanna)   
            continue
            
        # add bam and pairs to exp proc file
        list_release.extend([exp_com_bam[0],exp_pairs[0]])
        if add_pc:
            add_preliminary_processed_files(exp, [exp_com_bam[0],exp_pairs[0]], env)
        
        print exp, 'part2 complete'
        set_pairs.append(exp_pairs[0])
    
    if part3 != 'done':
        print 'Part3 not ready'
        continue
    
    if not set_pairs:
        print 'no pairs can be produced from this set'
        continue
        
    merged_pairs = []
    for set_pair in set_pairs:
        merged_pair = get_wfr_out(set_pair, 'hi-c-processing-pairs-nore 0.2.5', 'pairs', env)
        hic = get_wfr_out(set_pair, 'hi-c-processing-pairs-nore 0.2.5', 'hic', env)
        mcool = get_wfr_out(set_pair, 'hi-c-processing-pairs-nore 0.2.5', 'mcool', env)
        normvec = get_wfr_out(set_pair, 'hi-c-processing-pairs-nore 0.2.5', 'normvector_juicerformat', env)
        if merged_pair.startswith('no') or not merged_pair:
            part3 = 'not ready'
            break
        elif merged_pair == 'running':
            part3 = 'still running'
            break
        else:
            merged_pairs.append(merged_pair)
    
    
    # if part3 is still running report it, and skip the rest of the script
    if part3 == 'still running':
        print 'part3', part3
        continue        
                
    if part3 != 'done' or len(list(set(merged_pairs))) != 1:
        print a_set['accession'], 'is missing Part3'
        
        # if it is not run, and add_wfr is true, go for it, then skip the rest of the script
        if add_wfr:
            if not chrsize_ref:
                print 'not yet usable', organism
                continue

            inp_f = {'input_pairs':set_pairs, 'chromsizes':chrsize_ref} 
            run_missing_wfr(step_settings(2, organism), inp_f, a_set['accession'], env, tibanna)
        continue
    #####
    #add competed flag to experiment
    if add_tag:
        ff_utils.patch_metadata({"completed_processes":["HiC_Pipeline_0.2.5"]}, obj_id=a_set['accession'] , ff_env=env)
    
    # add processed files to set
    list_release.extend([merged_pair, hic, mcool, normvec])
    if add_pc:
        add_preliminary_processed_files(a_set['accession'], [merged_pair, hic, mcool], env)
    
    #release files and wfrs
    if add_rel:
        release_files(a_set['accession'], list(set(list_release)), env)
    
    completed += 1
    completed_acc.append(a_set['accession'])
    print a_set['accession'], 'part3 complete'
    


    
print completed
print completed_acc

14 total number of sets
1 sets completed

22
1 4DNESWWRKZDK
DNaseI mouse
4DNEXC8TTU1U part1 complete
4DNEXC8TTU1U part2 complete
4DNESWWRKZDK part3 complete

5
2 4DNESAGFJ56V skipping small file size 5

79
3 4DNES49IRGRY
DNaseI mouse
4DNEXYLI7G52 part1 complete
4DNEXYLI7G52 part2 complete
4DNES49IRGRY part3 complete

92
4 4DNESNPYVXDM
DNaseI mouse
4DNEXBH5B94I part1 complete
4DNEXBH5B94I part2 complete
4DNESNPYVXDM part3 complete

59
5 4DNESF8PGGTI
DNaseI mouse
4DNEX8XQA6B5 part1 complete
4DNEX8XQA6B5 part2 complete
4DNESF8PGGTI part3 complete

84
6 4DNESG15UT75
DNaseI mouse
4DNEX3IVMKFZ part1 complete
4DNEX3IVMKFZ part2 complete
4DNEXXX48QKL part1 complete
4DNEXXX48QKL part2 complete
4DNEXSURRHM7 part1 complete
4DNEXSURRHM7 part2 complete
4DNESG15UT75 part3 complete

42
7 4DNESZFHB53P
DNaseI human
4DNEX778XE4J part1 complete
4DNEX778XE4J part2 complete
4DNEXEKTHDKQ part1 complete
4DNEXEKTHDKQ part2 complete
4DNESZFHB53P part3 complete

39
8 4DNESGTHHJAC
DNaseI human
4DNEXGBN93KT part1

{'other_processed_files': [{'files': [u'/files-processed/4DNFIPCU2LGI/', u'/files-processed/4DNFI1KTTETW/'], 'type': 'preliminary', 'title': 'HiC Processing Pipeline - Preliminary Files'}]}
